In [ ]:
%load_ext autoreload
%autoreload 2

# Load a single ep and show some translations

In [ ]:
from pathlib import Path

from interleave_epub.epub.epub import EPub
from interleave_epub.nlp.local_spacy_model import spacy_load_local_model
from interleave_epub.nlp.cached_pipe import TranslationPipelineCache


### Load NLP models

In [ ]:
spacy_cache = Path("~/.cache/spacy_my_models").expanduser()

nlp = {
    "en": spacy_load_local_model("en_core_web_md", spacy_cache, force_download=False),
    "fr": spacy_load_local_model("fr_core_news_md", spacy_cache, force_download=False),
}


In [ ]:
hug_cache = Path("~/.cache/hug_my_trad").expanduser()
hug_trad_en = hug_cache / "translated_en_fr.json"
hug_trad_fr = hug_cache / "translated_fr_en.json"

pipe = {
    "en-fr": TranslationPipelineCache(None, hug_trad_en, "en-fr"),
    "fr-en": TranslationPipelineCache(None, hug_trad_fr, "fr-en"),
}

pipe["en-fr"]("Let's try this cool way to create a callable class.")

### Load Ep

In [ ]:
# a sample epub, in French
epub_folder_path = Path("~").expanduser() / "snippet" / "datasets" / "ebook"
an_epub_path = epub_folder_path / "Gaston_Leroux_-_Le_Mystere_de_la_chambre_jaune.epub"
an_epub_path


In [ ]:
an_epub = EPub(an_epub_path, "Chambre Jaune", "fr", "en", nlp, pipe)


In [ ]:
a_chap = an_epub.chapters[0]
a_par = a_chap.paragraphs[0]
a_par


## Immediately derail everything to pick better sentences

In [ ]:
a_par.par_doc.has_annotation("SENT_START")


In [ ]:
a_doc = a_par.par_doc.copy()


In [ ]:
for i_s, sents in enumerate(a_doc.sents):
    print(f"{i_s} {sents}")

In [ ]:
# https://spacy.io/api/doc#retokenize
# https://stackoverflow.com/questions/58294798/spacy-doc-merge-to-using-retokenizer
# https://stackoverflow.com/questions/65083559/how-to-write-code-to-merge-punctuations-and-phrases-using-spacy

start_it = 15
for it, token in enumerate(a_doc[start_it:25]):
    print(
        f"{it+start_it:3d} {token.text:15s}"
        f"{token.is_sent_start} {token.is_sent_end}"
    )


In [ ]:
with a_doc.retokenize() as retokenizer:
    attrs = {"LEMMA": "David Bowie"}
    retokenizer.merge(a_doc[1:2], attrs=attrs)
